**Angle Calculations for each sampling point in the mouse trajectory**
- write by XR @ 08/17/2021 based on 'ClusterRep_concentricCirCal.ipynb'
- In 'ClusterRep_concentricCirCal.ipynb', I did the concentric circle calculation
- step 1: normalize the mouse trajectory to 0-1
- step 2: interection point of the mouse trajectory and the circles with radius ranging from 0.02 to 0.98 in steps of 0.02

In [1]:
#%load_ext lab_black

In [1]:
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
import scipy
import sklearn
import os
import mne
import pycircstat as ps  # circular data analysis
import math

# another toolbox for circular data – "astropy package": from astropy import subpackage
from matplotlib import pyplot as plt
from scipy.io import loadmat

In [2]:
np.warnings.filterwarnings("ignore")

**自定义的函数**

In [3]:
def angle_to_onezero(tgtXY, zeroXY, centerXY):
    """
    calculate the angle between a target data point and zero point (centerX+RChunkPX, centerY)
    """
    tgtX, tgtY = tgtXY
    zeroX, zeroY = zeroXY
    centerX, centerY = centerXY
    Col1 = np.array([tgtX, tgtY]) - np.array([centerX, centerY])  # Col1, Col2 = (n,)
    Col2 = np.array([zeroX, zeroY]) - np.array([centerX, centerY])
    rad_t2o = np.arccos(
        np.dot(Col1, Col2) / (np.linalg.norm(Col1) * np.linalg.norm(Col2))
    )
    if tgtY <= centerY:
        rad_t2o = -rad_t2o
    else:
        rad_t2o = rad_t2o
    return rad_t2o

In [4]:
def node_which_cluster(nodeId, clsDef):
    """
    which cluster a specific node belongs to
    """
    cluster_in = np.zeros((np.shape(clsDef)[0], 1))
    for i in range(0, np.shape(clsDef)[0]):
        cluster_in[i] = np.sum(nodeId == clsDef[i, :])
    return cluster_in

In [5]:
def ismember(A, B):
    return [np.sum(a == B) for a in A]

In [6]:
def two_point_line(x1, y1, x2, y2):
    a = (y2 - y1) / (x2 - x1)  ## slope
    b = y1 - a * x1  ## intercept
    return a, b

# Experiment 1: implicit learning + mouse trajectory + sequential display

In [6]:
tgtNum = 1
dtrNums = np.array([1, 2, 3])
nodeNum = 15
clsDef = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]])

def LynnNetwork():
    transMat = np.zeros((nodeNum, nodeNum))
    transMat[np.array([2, 3, 4, 15]) - 1, 1 - 1] = 0.25
    transMat[np.array([1, 3, 4, 5]) - 1, 2 - 1] = 0.25
    transMat[np.array([1, 2, 4, 5]) - 1, 3 - 1] = 0.25
    transMat[np.array([1, 2, 3, 5]) - 1, 4 - 1] = 0.25
    transMat[np.array([2, 3, 4, 6]) - 1, 5 - 1] = 0.25

    transMat[np.array([5, 7, 8, 9]) - 1, 6 - 1] = 0.25
    transMat[np.array([6, 8, 9, 10]) - 1, 7 - 1] = 0.25
    transMat[np.array([6, 7, 9, 10]) - 1, 8 - 1] = 0.25
    transMat[np.array([6, 7, 8, 10]) - 1, 9 - 1] = 0.25
    transMat[np.array([7, 8, 9, 11]) - 1, 10 - 1] = 0.25

    transMat[np.array([10, 12, 13, 14]) - 1, 11 - 1] = 0.25
    transMat[np.array([11, 13, 14, 15]) - 1, 12 - 1] = 0.25
    transMat[np.array([11, 12, 14, 15]) - 1, 13 - 1] = 0.25
    transMat[np.array([11, 12, 13, 15]) - 1, 14 - 1] = 0.25
    transMat[np.array([1, 12, 13, 14]) - 1, 15 - 1] = 0.25
    return transMat

transMat = LynnNetwork()

**Experimental Settings**

In [ ]:
# Experiment settings: iMac
sampleRate  = 60
scrWidthCM  = 47.9
scrHeightCM = 26.9
scrWidthPX  = 2048
scrHeightPX = 1152
centerX     = scrWidthPX / 2
centerY     = scrHeightPX / 2
visualAngle    = 9
visualAngleCir = (visualAngle / 180) * np.pi
visualDist     = 86
dispDist       = 2 * visualDist * np.tan(visualAngleCir / 2)
RChunk   = dispDist / 2  # radias of the invisible circle
cm2pxV   = scrHeightPX / scrHeightCM
cm2pxH   = scrWidthPX / scrWidthCM
RChunkPX = RChunk * cm2pxV
zeroPos  = np.array([centerX + RChunkPX, centerY + 0])
# (2) time settings
fixationDur = 0.6
respDur     = 2.0
grayoffT    = 0.8
grayOnT     = 0.8
bonusDec    = 0.5
bonusTime   = 1
# (3) trial settings
rndTrial = 700
HamTrial = 800
nTrials  = rndTrial + HamTrial
rndInter = 85
HamInter = 15
# experiment 1.1-1.3
# rndArray = np.array([230, 2 * 230, 2 * 230 + 240], dtype=int)  # ..................The 1st 3 blocks were random pathways
# experiment 1.4
rndArray = np.array([200, 2 * 200, 3 * 200, 3 * 200 + 100], dtype=int)
HamArray = np.asarray(np.array(np.linspace(700 + 200, 1500, 4, dtype=int)))  # The last 4 blocks were random + hamiltonian mixtures; !! np.asarray(): convert a sequence into array !!
tInBlc   = np.concatenate((rndArray, HamArray))
nBlock      = len(tInBlc)
trialsInBlc = np.zeros((nBlock, 2), dtype=int)
trialsInBlc[0, 0]  = 1 - 1
trialsInBlc[1:, 0] = tInBlc[:-1]
trialsInBlc[:, 1]  = tInBlc

# Behavioral data column index
rndOrHam = 0  # trial countings for the random (1-700) and hamiltonian (1-800) walk
blockNo  = 1  # block number
trialNo  = 2  # trial number
trialTp  = 3  # trial type: 1-random; 2-hamiltonian
objTgt   = 4  # target object
objResp  = np.array([5, 6, 7, 8])  # responded objects
moveTime = 9  # moving time of the mouse
respRT   = 10  # response reaction time
dispRT   = 11  # object total display time
errorIdx = 12  # whether error (including incorrect response and time out) or not: 1-yes
timeOIdx = 13  # whether time out: 1-yes
bonusCn  = 14  # bonus in each trial
trialLt  = 15  # trial total length
imgPosX  = np.array([16, 17, 18, 19])
imgPosY  = np.array([20, 21, 22, 23])
fastIdx  = 24

folder   = '/Users/ren/Projects-NeuroCode/MyExperiment/HierarchicalCluster'

In [9]:
# Read the mouse trajectory for each block
data_behv = folder + '/FormalExp-LynnNetwork-Results/' 
subj_list = [
    "wsn_1_f_18",
    "dy_2_f_22",
    "haq_3_f_24",
    "hry_4_f_20",
    "zjx_5_m_20",
    "yyq_6_f_18",
    "zkw_7_m_18",
    "zy_8_f_20",
    "hys_9_m_20",
    "cjj_10_m_18",
    "dwq_11_f_22",
    "ljl_12_m_20",
    "jyx_13_m_19",
    "zk_14_f_21",
    "lsy_15_m_19",
    "cjl_16_m_19",
    "yjy_17_f_23",
    "lym_18_f_19",
    "pr_19_f_23",
    "ws_20_f_21",
    "wn_21_f_21",
    "hjy_22_f_18",
    "qyk_23_f_22",
    "yd_24_f_20",
]
subjLab = [
    "wsn1",
    "dy2",
    "haq3",
    "hry4",
    "zjx5",
    "yyq6",
    "zkw7",
    "zy8",
    "hys9",
    "cjj10",
    "dwq11",
    "ljl12",
    "jyx13",
    "zk14",
    "lsy15",
    "cjl16",
    "yjy17",
    "lym18",
    "pr19",
    "ws20",
    "wn21",
    "hjy22",
    "qyk23",
    "yd24",
]
subj_listBv = subj_list
subLen = len(subj_list)

path_or_buf = data_behv + "clusterResult_pd.csv"
clusterResult_pd = pd.read_csv(filepath_or_buffer=path_or_buf)
clusterResult_pd = clusterResult_pd.drop(columns=["Unnamed: 0"])
clusterResult_pd

,trialCnt,blockNo,trialNo,trialType,targetObj,respObj0,respObj1,respObj2,respObj3,moveTime,...,fromNd,toNd,subNo,dtrIn,dtrNoCnt,transType,trialMark,trialMark2,recencyL,recencySL
0,1,1,1,1,1,3,14,0,0,0.000152,...,1,0,0,0,1,2,0,1,15,15
1,2,1,2,1,3,5,12,9,0,0.600372,...,0,1,0,0,2,1,0,1,15,15
2,3,1,3,1,5,6,7,11,10,0.000190,...,1,1,0,0,3,3,0,1,15,15
3,4,1,4,1,6,5,13,0,0,0.350270,...,1,1,0,0,1,3,0,1,15,15
4,5,1,5,1,5,6,15,10,14,0.350192,...,1,1,0,0,3,3,0,1,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,796,7,196,2,12,14,1,0,0,0.049220,...,0,0,23,0,1,0,0,0,42,44
35996,797,7,197,2,14,13,8,0,0,0.066696,...,0,0,23,0,1,0,0,0,42,46
35997,798,7,198,2,13,15,10,1,9,0.167739,...,0,1,23,0,3,1,0,1,42,51
35998,799,7,199,2,15,1,4,0,0,0.266468,...,1,1,23,0,1,3,0,1,59,63


In [10]:
clusterResult_pd['RT'].min(), clusterResult_pd['RT'].max()

(0.3835245410000425, 2.12423131200012)

# Experiment 2: explicit learning + mouse trajectory + sequential display

In [9]:
data_behv = folder + '/FormalExp-ExplicitLearning-LynnNetwork-Results/' 
subj_list = [
    "hsp_1_m_21",
    "hr_2_f_22",
    "pxy_3_m_19",
    "wyx_4_m_22",
    "lml_5_f_19",
    "lf_6_m_21",
    "md_7_f_21",
    "srz_8_m_20",
    "fsq_9_f_18",
    "caq_10_f_20",
    "xjm_11_f_26",
    "cjs_12_m_19",
    "xxx_13_f_19",
    "wsq_14_f_19",
    "zzm_15_f_18",
    "lsy_16_f_20",
    "man_17_f_19",
    "zxy_18_f_18",
    "cyh_19_f_21",
    "szn_20_f_19",
    "sb_21_m_22",
    "drq_22_f_18",
    "cy_23_f_18",
    "zhc_24_m_21",
]
subjLab = [
    "hsp1",
    "hr2",
    "pxy3",
    "wyx4",
    "lml5",
    "lf6",
    "md7",
    "srz8",
    "fsq9",
    "caq10",
    "xjm11",
    "cjs12",
    "xxx13",
    "wsq14",
    "zzm15",
    "lsy16",
    "man17",
    "zxy18",
    "cyh19",
    "szn20",
    "sb21",
    "drq22",
    "cy23",
    "zhc24",
]
subj_listBv = subj_list
subLen = len(subj_list)

path_or_buf = data_behv + "clusterResult_pd.csv"
clusterResult_pd = pd.read_csv(filepath_or_buffer=path_or_buf)
clusterResult_pd = clusterResult_pd.drop(columns=["Unnamed: 0"])
clusterResult_pd

,trialCnt,blockNo,trialNo,trialType,targetObj,respObj0,respObj1,respObj2,respObj3,moveTime,...,fromNd,toNd,subNo,dtrIn,dtrNoCnt,transType,trialMark,trialMark2,recencyL,recencySL
0,1,1,1,1,15,14,5,6,0,0.249359,...,1,0,0,0,2,2,-1,2,15,15
1,2,1,2,1,14,13,5,7,9,0.000143,...,0,0,0,0,3,0,-1,3,15,15
2,3,1,3,1,13,12,5,0,0,0.000118,...,0,0,0,0,1,0,0,1,15,15
3,4,1,4,1,12,14,7,0,0,0.000095,...,0,0,0,0,1,0,-1,3,15,15
4,5,1,5,1,14,11,8,0,0,0.149780,...,0,1,0,0,1,1,-1,3,3,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,796,7,196,2,3,4,11,0,0,0.000104,...,0,0,23,0,1,0,0,1,21,24
35996,797,7,197,2,4,1,13,0,0,0.047769,...,0,1,23,0,1,1,0,1,18,21
35997,798,7,198,2,1,15,5,14,0,0.000092,...,1,1,23,1,2,3,-1,2,17,25
35998,799,7,199,2,15,14,2,0,0,0.000100,...,1,0,23,0,1,2,0,1,14,17


In [12]:
clusterResult_pd['RT'].min(), clusterResult_pd['RT'].max()

(0.4157223850197624, 2.211525561999224)

# Experiment 3: implicit learning + mouse trajectory + shuffled display for Random Walk

In [9]:
data_behv = folder + '/FormalExp-LynnNetwork-ImplicitRandom-Results/' 
subj_list = [
    "zyh_1_f_21",
    "why_2_m_18",
    "cr_3_f_19",
    "zyx_4_f_25",
    "wym_5_m_23",
    "wd_6_f_18",
    "lyh_7_f_21",
    "zr_8_f_24",
    "zyh_9_m_19",
    "zzy_10_f_19",
    "smq_11_f_25",
    "sz_12_f_20",
    "lzy_13_m_22",
    "yxy_14_f_18",
    "zxl_15_f_18",
    "wqh_16_m_23",
    "zxj_17_m_21",
    "skx_18_f_20",
    "zlh_19_m_24",
    "gwt_20_f_23",
    "lwn_21_f_18",
    "lrp_22_m_21",
    "sjj_23_f_18",
    "xy_24_f_19",
]
subjLab = [
    "zyh1",
    "why2",
    "cr3",
    "zyx4",
    "wym5",
    "wd6",
    "lyh7",
    "zr8",
    "lyh9",
    "zzy10",
    "smq11",
    "sz12",
    "lzy13",
    "yxy14",
    "zxl15",
    "wqh16",
    "zxj17",
    "skx18",
    "zlh19",
    "gwt20",
    "lwn21",
    "lrp22",
    "sjj23",
    "xy24",
]
subj_listBv = subj_list
subLen = len(subj_list)

path_or_buf = data_behv + "clusterResult_pd.csv"
clusterResult_pd = pd.read_csv(filepath_or_buffer=path_or_buf)
clusterResult_pd = clusterResult_pd.drop(columns=["Unnamed: 0"])
clusterResult_pd

,trialCnt,blockNo,trialNo,trialType,targetObj,respObj0,respObj1,respObj2,respObj3,moveTime,...,fromNd,toNd,subNo,dtrIn,dtrNoCnt,transType,trialMark,trialMark2,recencyL,recencySL
0,1,1,1,1,11,12,5,0,0,0.100143,...,1,0,0,0,1,2,0,1,15,15
1,2,1,2,1,13,11,5,0,0,0.083442,...,0,1,0,0,1,1,0,1,15,15
2,3,1,3,1,9,6,3,2,5,0.000137,...,0,1,0,0,3,1,0,1,15,15
3,4,1,4,1,9,8,3,0,0,0.283568,...,0,0,0,0,1,0,0,1,1,15
4,5,1,5,1,11,10,4,6,7,0.000190,...,1,1,0,0,3,3,0,1,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,796,7,196,2,8,9,15,4,0,0.000116,...,0,0,23,0,2,0,0,1,12,45
35996,797,7,197,2,9,7,4,2,15,0.000154,...,0,0,23,0,3,0,0,1,44,69
35997,798,7,198,2,7,10,13,0,0,0.033359,...,0,1,23,0,1,1,0,1,18,19
35998,799,7,199,2,10,11,3,0,0,0.000168,...,1,1,23,0,1,3,0,1,16,31


In [ ]:
clusterResult_pd['RT'].min(), clusterResult_pd['RT'].max()

# Compute, for each sampled point along the mouse trajectory, the angle between the mouse position and the target image or the distractor image, as well as the distance the mouse moves toward each image


**Description of the computation process**
- Compute, for each sampled point along the mouse trajectory, the angle between the mouse position and the target image or the distractor image.

In [11]:
np.warnings.filterwarnings("ignore")
expWord = 'implicitRandomCluster'
for SubIdx in range(0, subLen):
    subjBv = subj_listBv[SubIdx]
    subTit = subjLab[SubIdx]
    subjDir = data_behv + subjBv + "/"
    #indexMat    = loadmat(subjDir + subjBv + 'trialsInBlc.mat') 
    #trialsInBlc = indexMat['trialsInBlc']
    #trialsInBlc = trialsInBlc - 1
    
    for iBlock in range(0, nBlock):
        trialCount_iBlock = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "trialCnt"]
        trialType_iBlock  = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "trialType"]
        targetObj_iBlock  = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "targetObj"]
        respObj0_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "respObj0"]
        respObj1_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "respObj1"]
        respObj2_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "respObj2"]
        respObj3_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "respObj3"]
        moveTime_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "moveTime"]
        RT_iBlock         = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "RT"]
        dispRT_iBlock     = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "dispRT"]
        errIdx_iBlock     = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "errorIdx"]
        timeOutIdx_iBlock = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "timeOutIdx"]
        bonus_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "bonus"]
        trialLt_iBlock    = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "trialLt"]
        ObjX0_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjX0"]
        ObjX1_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjX1"]
        ObjX2_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjX2"]
        ObjX3_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjX3"]
        ObjY0_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjY0"]
        ObjY1_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjY1"]
        ObjY2_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjY2"]
        ObjY3_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjY3"]
        fastIdx_iBlock    = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "fastIdx"]
        fromNd_iBlock     = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "fromNd"]
        toNd_iBlock       = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "toNd"]
        dtrIn_iBlock      = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "dtrIn"]
        dtrNoCnt_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "dtrNoCnt"]
        transType_iBlock  = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "transType"]
        trialMark_iBlock  = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "trialMark"]
        trialMark2_iBlock = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "trialMark2"]
        recencyL_iBlock   = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "recencyL"]
        recencySL_iBlock  = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "recencySL"]
        tgtX_blc          = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjX0"]
        tgtY_blc          = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), "ObjY0"]
        respX_blc         = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), ["ObjX1", "ObjX2", "ObjX3"]]
        respY_blc         = clusterResult_pd.loc[(clusterResult_pd["blockNo"] == (iBlock + 1)) & (clusterResult_pd["subNo"] == SubIdx), ["ObjY1", "ObjY2", "ObjY3"]]
        
        iBlock_use = iBlock
            
        ## load the mouseTraj_trials.m
        mouseTraj_cell   = loadmat(subjDir + subjBv + "mouseTraj_trials_in_Blc" + str(iBlock_use + 1) + ".mat")
        ## load the respYes_trials.m
        respYes_cell     = loadmat(subjDir + subjBv + "respYes_trials_in_Blc" + str(iBlock_use + 1) + ".mat")
        mouseTraj_trials = mouseTraj_cell["mouseTraj_trials"]  # np.shape(mouseTraj_trials)
        respYes_trials   = respYes_cell["respYes_trials"]

        # loop across the trials
        for iT in range(0, np.shape(mouseTraj_trials)[0]):
            dtrNo_i        = dtrNoCnt_iBlock.iloc[iT]
            mouseTraj_iT   = mouseTraj_trials[iT][0]
            mouseTraj_arr  = mouseTraj_iT[:, [0, 1]]  # x-pos and y-pos
            mouseTraj_tm   = mouseTraj_iT[:, 3]  # time-pass
            mouseTraj_norm = mouseTraj_arr
            # the target
            tgtX_norm = tgtX_blc.iloc[iT]
            tgtY_norm = tgtY_blc.iloc[iT]
            # the distractors
            respX_i = respX_blc.iloc[iT, :]
            respY_i = respY_blc.iloc[iT, :]
            respX_norm = respX_i
            respY_norm = respY_i

            sampleLen  = np.shape(mouseTraj_norm)[0]  # sampling numbers
            radius_col = np.zeros((sampleLen, 1))  # radius of each point in mouse trajectory
            deltas_col = np.zeros((sampleLen, 1))  # intersection angle between origin-to-target and origin-to-intersection
            deltas_dtr = np.zeros((sampleLen, max(dtrNums)))  # intersection angle between origin-to-distractor and origin-to-intersection
            for iR in range(0, sampleLen):
                # calculate the angle between origin-to-target and origin-to-SamplingPoint
                # firstly, calculate the angle between target stimuli and the zeroPos
                tgtXY    = np.array([tgtX_norm, tgtY_norm])
                zeroXY   = np.array([zeroPos[0], zeroPos[1]])
                centerXY = np.array([centerX, centerY])
                rad_t2o  = angle_to_onezero(tgtXY, zeroXY, centerXY)
                # secondly, calculate the angle between mouse trajectory sample and the zeroPos
                mouXY    = np.array([mouseTraj_norm[iR, 0], mouseTraj_norm[iR, 1]])
                zeroXY   = np.array([zeroPos[0], zeroPos[1]])
                centerXY = np.array([centerX, centerY])
                # Sometimes the code output: 'RuntimeWarning: invalid value encountered in double_scalars', this is because Col2 is exactly at the (centerX, centerY)
                rad_m2o        = angle_to_onezero(mouXY, zeroXY, centerXY)
                deltas_col[iR] = rad_m2o - rad_t2o
                # distance between mouse trajectory and center point
                Xdist_iD = mouseTraj_norm[iR, 0] - centerX
                Ydist_iD = mouseTraj_norm[iR, 1] - centerY
                XYdist_iD = np.squeeze([Xdist_iD, Ydist_iD]).T
                radius_col[iR] = np.linalg.norm(XYdist_iD)

                # calculate the angle between origin-to-distractor and origin-to-SamplingPoint
                for iD in range(dtrNo_i):
                    dtrXY     = np.array([respX_norm[iD], respY_norm[iD]])
                    zeroXY    = np.array([zeroPos[0], zeroPos[1]])
                    centerXY  = np.array([centerX, centerY])
                    rad_d2o   = angle_to_onezero(dtrXY, zeroXY, centerXY)
                    rad_d2oDf = rad_m2o - rad_d2o
                    if rad_d2oDf < -np.pi:
                        rad_d2oDf = 2 * np.pi + rad_d2oDf
                    elif rad_d2oDf > np.pi:
                        rad_d2oDf = -2 * np.pi + rad_d2oDf
                    deltas_dtr[iR, iD] = rad_d2oDf

            deltas_col[deltas_col < -np.pi] = (2 * np.pi + deltas_col[deltas_col < -np.pi])
            deltas_col[deltas_col > np.pi]  = -2 * np.pi + deltas_col[deltas_col > np.pi]

            ## calculate the accumulated moving distance towards target or distractors
            # added by rxj @ 08/19/2021
            deltas_all = np.concatenate([deltas_col, deltas_dtr], axis=1)
            apprWhich  = np.empty((sampleLen, 1 + max(dtrNums)))  # at the current moment, which object the subjects was approaching
            apprWhich[:] = np.NaN
            for iR in range(0, sampleLen):
                deltas_iR = deltas_all[iR, :]
                if ~(np.isnan(deltas_iR).any()):
                    apprWhich[iR, :] = 0
                    minId = np.argmin(np.abs(deltas_iR[: (1 + dtrNo_i)]))
                    apprWhich[iR, minId] = 1

            accmDis = np.zeros((sampleLen, 1 + max(dtrNums)))
            used_iR = []
            for iR in range(0, sampleLen):
                apprWhich_iR = apprWhich[iR, :]
                if (~(np.isnan(apprWhich_iR).any())) and (np.sum(apprWhich_iR) == 1):
                    mouX = mouseTraj_norm[iR, 0]
                    mouY = mouseTraj_norm[iR, 1]
                    if (iR == 0) or (used_iR == []):
                        accmXY_iR = np.squeeze([mouX - centerX, mouY - centerY]).T
                    else:
                        mouX_t = mouseTraj_norm[used_iR[len(used_iR) - 1], 0]
                        mouY_t = mouseTraj_norm[used_iR[len(used_iR) - 1], 1]
                        accmXY_iR = np.squeeze([mouX - mouX_t, mouY - mouY_t]).T

                    used_iR.append(iR)
                    accmDis[iR, apprWhich_iR == 1] = np.linalg.norm(accmXY_iR)
            ## convert to cumulative sum
            cumsumDis = np.cumsum(accmDis, axis=0)
            ## instaneous speed
            instSpd   = accmDis / (1 / sampleRate)

            ## prepare columns for the data
            subjNo_col    = np.asarray([SubIdx] * sampleLen).reshape((sampleLen, 1))
            blockNo_col   = np.asarray([iBlock] * sampleLen).reshape((sampleLen, 1))
            trialNo_col   = np.asarray([iT] * sampleLen).reshape((sampleLen, 1))
            sampleCnt_col = np.asarray(range(0, sampleLen)).reshape((sampleLen, 1))
            tgtX_norm_col = np.asarray([tgtX_norm] * sampleLen).reshape((sampleLen, 1))
            tgtY_norm_col = np.asarray([tgtY_norm] * sampleLen).reshape((sampleLen, 1))
            dtrX_norm_col = np.asarray([respX_norm] * sampleLen)
            dtrY_norm_col = np.asarray([respY_norm] * sampleLen)

            trialCount_col = np.asarray([trialCount_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            trialType_col  = np.asarray([trialType_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            targetObj_col  = np.asarray([targetObj_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            respObj0_col   = np.asarray([respObj0_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            respObj1_col   = np.asarray([respObj1_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            respObj2_col   = np.asarray([respObj2_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            respObj3_col   = np.asarray([respObj3_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            moveTime_col   = np.asarray([moveTime_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            RT_col         = np.asarray([RT_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            dispRT_col     = np.asarray([dispRT_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            errIdx_col     = np.asarray([errIdx_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            timeOutIdx_col = np.asarray([timeOutIdx_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            bonus_col      = np.asarray([bonus_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            trialLt_col    = np.asarray([trialLt_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjX0_col      = np.asarray([ObjX0_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjX1_col      = np.asarray([ObjX1_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjX2_col      = np.asarray([ObjX2_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjX3_col      = np.asarray([ObjX3_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjY0_col      = np.asarray([ObjY0_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjY1_col      = np.asarray([ObjY1_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjY2_col      = np.asarray([ObjY2_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            ObjY3_col      = np.asarray([ObjY3_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            fastIdx_col    = np.asarray([fastIdx_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            fromNd_col     = np.asarray([fromNd_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            toNd_col       = np.asarray([toNd_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            dtrIn_col      = np.asarray([dtrIn_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            dtrNoCnt_col   = np.asarray([dtrNoCnt_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            transType_col  = np.asarray([transType_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            trialMark_col  = np.asarray([trialMark_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            trialMark2_col = np.asarray([trialMark2_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            recencyL_col   = np.asarray([recencyL_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            recencySL_col  = np.asarray([recencySL_iBlock.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            # label trials with RT smaller than 0.8s
            if RT_iBlock.iloc[iT] <= grayoffT:
                preResp_col = np.ones((sampleLen, 1))
            else:
                preResp_col = np.zeros((sampleLen, 1))
            # Labeling the trials where participants make incorrect response before cue appears
            mouse_bfCue = np.sum(mouseTraj_tm <= grayoffT)
            respYes_arr = respYes_trials[iT][0][:mouse_bfCue]
            respIncor   = np.sum(respYes_arr == -1)
            if respIncor >= 1:
                corrBfCue = -1  # tried to respond before cue appears, but failed
            else:
                corrBfCue = 0
            corrBfCueC_col = np.asarray([corrBfCue] * sampleLen).reshape((sampleLen, 1))

            # Concatenate the columns
            intXY_col = mouseTraj_norm
            angleTraj_conc = np.concatenate(
                [
                    subjNo_col,
                    trialCount_col,
                    blockNo_col,
                    trialNo_col,
                    trialType_col,
                    sampleCnt_col,
                    radius_col,
                    intXY_col,
                    deltas_col,
                    deltas_dtr,
                    accmDis,
                    cumsumDis,
                    instSpd,
                    tgtX_norm_col,
                    tgtY_norm_col,
                    dtrX_norm_col,
                    dtrY_norm_col,
                    targetObj_col,
                    respObj0_col,
                    respObj1_col,
                    respObj2_col,
                    respObj3_col,
                    RT_col,
                    errIdx_col,
                    timeOutIdx_col,
                    fastIdx_col,
                    fromNd_col,
                    toNd_col,
                    dtrIn_col,
                    dtrNoCnt_col,
                    transType_col,
                    trialMark_col,
                    trialMark2_col,
                    preResp_col,
                    corrBfCueC_col,
                    recencyL_col,
                    recencySL_col,
                ],
                axis=1,
            )
            if iT == 0 and iBlock == 0:
                angleTraj_pd = pd.DataFrame(
                    angleTraj_conc,
                    columns=[
                        "subNo",
                        "trialCnt",
                        "blockNo",
                        "trialNo",
                        "trialType",
                        "sampleCnt",
                        "radiusCon",
                        "crossX",
                        "crossY",
                        "tgtAng",
                        "dtrAng1",
                        "dtrAng2",
                        "dtrAng3",
                        "tgtDist",
                        "dtrDist1",
                        "dtrDist2",
                        "dtrDist3",
                        "tgtCumSum",
                        "dtrCumSum1",
                        "dtrCumSum2",
                        "dtrCumSum3",
                        "tgtSpd",
                        "dtrSpd1",
                        "dtrSpd2",
                        "dtrSpd3",
                        "tgtXnorm",
                        "tgtYnorm",
                        "dtrXnorm1",
                        "dtrXnorm2",
                        "dtrXnorm3",
                        "dtrYnorm1",
                        "dtrYnorm2",
                        "dtrYnorm3",
                        "targetObj",
                        "respObj0",
                        "respObj1",
                        "respObj2",
                        "respObj3",
                        "RT",
                        "errorIdx",
                        "timeOutIdx",
                        "fastIdx",
                        "fromNd",
                        "toNd",
                        "dtrIn",
                        "dtrNoCnt",
                        "transType",
                        "trialMark",
                        "trialMark2",
                        "preResp",
                        "respBfCue",
                        "recencyL",
                        "recencySL",
                    ],
                )

            else:
                angleTraj_pd2 = pd.DataFrame(
                    angleTraj_conc,
                    columns=[
                        "subNo",
                        "trialCnt",
                        "blockNo",
                        "trialNo",
                        "trialType",
                        "sampleCnt",
                        "radiusCon",
                        "crossX",
                        "crossY",
                        "tgtAng",
                        "dtrAng1",
                        "dtrAng2",
                        "dtrAng3",
                        "tgtDist",
                        "dtrDist1",
                        "dtrDist2",
                        "dtrDist3",
                        "tgtCumSum",
                        "dtrCumSum1",
                        "dtrCumSum2",
                        "dtrCumSum3",
                        "tgtSpd",
                        "dtrSpd1",
                        "dtrSpd2",
                        "dtrSpd3",
                        "tgtXnorm",
                        "tgtYnorm",
                        "dtrXnorm1",
                        "dtrXnorm2",
                        "dtrXnorm3",
                        "dtrYnorm1",
                        "dtrYnorm2",
                        "dtrYnorm3",
                        "targetObj",
                        "respObj0",
                        "respObj1",
                        "respObj2",
                        "respObj3",
                        "RT",
                        "errorIdx",
                        "timeOutIdx",
                        "fastIdx",
                        "fromNd",
                        "toNd",
                        "dtrIn",
                        "dtrNoCnt",
                        "transType",
                        "trialMark",
                        "trialMark2",
                        "preResp",
                        "respBfCue",
                        "recencyL",
                        "recencySL",
                    ],
                )

                angleTraj_pd = angleTraj_pd.append(angleTraj_pd2)

    # -------------------------------------------------------------------
    angleTraj_pd.subNo      = angleTraj_pd.subNo.astype("int64")
    angleTraj_pd.trialCnt   = angleTraj_pd.trialCnt.astype("int64")
    angleTraj_pd.blockNo    = angleTraj_pd.blockNo.astype("int64")
    angleTraj_pd.trialNo    = angleTraj_pd.trialNo.astype("int64")
    angleTraj_pd.trialType  = angleTraj_pd.trialType.astype("int64")
    angleTraj_pd.sampleCnt  = angleTraj_pd.sampleCnt.astype("int64")
    angleTraj_pd.targetObj  = angleTraj_pd.targetObj.astype("int64")
    angleTraj_pd.respObj0   = angleTraj_pd.respObj0.astype("int64")
    angleTraj_pd.respObj1   = angleTraj_pd.respObj1.astype("int64")
    angleTraj_pd.respObj2   = angleTraj_pd.respObj2.astype("int64")
    angleTraj_pd.respObj3   = angleTraj_pd.respObj3.astype("int64")
    angleTraj_pd.errorIdx   = angleTraj_pd.errorIdx.astype("int64")
    angleTraj_pd.timeOutIdx = angleTraj_pd.timeOutIdx.astype("int64")
    angleTraj_pd.fastIdx    = angleTraj_pd.fastIdx.astype("int64")
    angleTraj_pd.fromNd     = angleTraj_pd.fromNd.astype("int64")
    angleTraj_pd.toNd       = angleTraj_pd.toNd.astype("int64")
    angleTraj_pd.dtrIn      = angleTraj_pd.dtrIn.astype("int64")
    angleTraj_pd.dtrNoCnt   = angleTraj_pd.dtrNoCnt.astype("int64")
    angleTraj_pd.transType  = angleTraj_pd.transType.astype("int64")
    angleTraj_pd.trialMark  = angleTraj_pd.trialMark.astype("int64")
    angleTraj_pd.trialMark2 = angleTraj_pd.trialMark2.astype("int64")
    angleTraj_pd.preResp    = angleTraj_pd.preResp.astype("int64")
    angleTraj_pd.respBfCue  = angleTraj_pd.respBfCue.astype("int64")
    angleTraj_pd.recencyL   = angleTraj_pd.recencyL.astype("int64")
    angleTraj_pd.recencySL  = angleTraj_pd.recencySL.astype("int64")

    # save the dataframe for each subjecta: df.to_save(r'Path where you want to store the exported CSV file/File Name.csv', index=False); if you wish to include the index, then simply remove "index=False" from the code
    path_or_buf = subjDir + subjBv + "angleTraj_pd.csv"
    angleTraj_pd.to_csv(path_or_buf=path_or_buf)

    del angleTraj_pd, angleTraj_pd2

**Concatenate data from all participants**

In [12]:
for SubIdx in range(0, subLen):
    subjBv = subj_listBv[SubIdx]
    subTit = subjLab[SubIdx]
    subjDir = data_behv + subjBv + "/"
    path_or_buf = subjDir + subjBv + "angleTraj_pd.csv"
    if SubIdx == 0:
        angleTraj_pd_subj = pd.read_csv(filepath_or_buffer=path_or_buf, index_col=0)  # ignore the untitled 1st colimn
    else:
        angleTraj_pd_tmp = pd.read_csv(filepath_or_buffer=path_or_buf, index_col=0)  # ignore the untitled 1st colimn
        angleTraj_pd_subj = angleTraj_pd_subj.append(angleTraj_pd_tmp)

re_name_cluster = {1: "cluster1", 2: "cluster1", 3: "cluster1", 4: "cluster1", 5: "cluster1", 
                   6: "cluster2", 7: "cluster2", 8: "cluster2", 9: "cluster2", 10: "cluster2",
                   11: "cluster3", 12: "cluster3", 13: "cluster3", 14: "cluster3", 15: "cluster3"}
angleTraj_pd_subj["clsNameFr"] = angleTraj_pd_subj["targetObj"]
angleTraj_pd_subj["clsNameTo"] = angleTraj_pd_subj["respObj0"]
angleTraj_pd_subj = angleTraj_pd_subj.replace({"clsNameFr": re_name_cluster})
angleTraj_pd_subj = angleTraj_pd_subj.replace({"clsNameTo": re_name_cluster})
re_name_nodeId = {1: "out", 2: "in", 3: "in", 4: "in", 5: "out",
                  6: "out", 7: "in", 8: "in", 9: "in", 10: "out",
                  11: "out", 12: "in", 13: "in", 14: "in", 15: "out"}
angleTraj_pd_subj["nodeIdFr"] = angleTraj_pd_subj["targetObj"]
angleTraj_pd_subj["nodeIdTo"] = angleTraj_pd_subj["respObj0"]
angleTraj_pd_subj = angleTraj_pd_subj.replace({"nodeIdFr": re_name_nodeId})
angleTraj_pd_subj = angleTraj_pd_subj.replace({"nodeIdTo": re_name_nodeId})
re_name_nodeCat = {1: "1", 2: "2", 3: "3", 4: "4", 5: "5",
                   6: "6", 7: "7", 8: "8", 9: "9", 10: "10",
                   11: "11", 12: "12", 13: "13", 14: "14", 15: "15"}
angleTraj_pd_subj["nodeCatFr"] = angleTraj_pd_subj["targetObj"]
angleTraj_pd_subj["nodeCatTo"] = angleTraj_pd_subj["respObj0"]
angleTraj_pd_subj = angleTraj_pd_subj.replace({"nodeCatFr": re_name_nodeCat})
angleTraj_pd_subj = angleTraj_pd_subj.replace({"nodeCatTo": re_name_nodeCat})

# concentricCir_pd_subj = concentricCir_pd_subj.drop(columns=['index'])
path_or_buf = data_behv + "angleCal/" + "angleTraj_pd_subj.csv"
angleTraj_pd_subj.to_csv(path_or_buf=path_or_buf)

In [13]:
path_or_buf = data_behv + "angleCal/" + "angleTraj_pd_subj.csv"
angleTraj_pd_subj = pd.read_csv(filepath_or_buffer=path_or_buf)
angleTraj_pd_subj = angleTraj_pd_subj.drop(columns=["Unnamed: 0"])
angleTraj_pd_subj

,subNo,trialCnt,blockNo,trialNo,trialType,sampleCnt,radiusCon,crossX,crossY,tgtAng,...,preResp,respBfCue,recencyL,recencySL,clsNameFr,clsNameTo,nodeIdFr,nodeIdTo,nodeCatFr,nodeCatTo
0,0,1,0,0,1,0,0.000000,1024.0,576.0,NaN,...,0,0,15,15,cluster2,cluster2,out,in,6,8
1,0,1,0,0,1,1,0.000000,1024.0,576.0,NaN,...,0,0,15,15,cluster2,cluster2,out,in,6,8
2,0,1,0,0,1,2,0.000000,1024.0,576.0,NaN,...,0,0,15,15,cluster2,cluster2,out,in,6,8
3,0,1,0,0,1,3,0.000000,1024.0,576.0,NaN,...,0,0,15,15,cluster2,cluster2,out,in,6,8
4,0,1,0,0,1,4,0.000000,1024.0,576.0,NaN,...,0,0,15,15,cluster2,cluster2,out,in,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437051,12,800,7,199,2,100,307.278375,1252.0,782.0,0.028497,...,0,0,16,32,cluster3,cluster2,out,out,11,10
1437052,12,800,7,199,2,101,293.981292,1244.0,771.0,0.018976,...,0,0,16,32,cluster3,cluster2,out,out,11,10
1437053,12,800,7,199,2,102,290.496127,1242.0,768.0,0.015815,...,0,0,16,32,cluster3,cluster2,out,out,11,10
1437054,12,800,7,199,2,103,290.496127,1242.0,768.0,0.015815,...,0,0,16,32,cluster3,cluster2,out,out,11,10


## Read the data at each time point and save it as a separate file for subsequent LMM analyses

### Merge all trials

In [12]:
angleTraj_pd_corr = angleTraj_pd_subj
angleTraj_pd_corr.loc[angleTraj_pd_corr["blockNo"] >= 3, "trialCnt"] = (angleTraj_pd_corr.loc[angleTraj_pd_corr["blockNo"] >= 3, "trialCnt"] + 700)
re_name_trialType = {1: "random", 2: "hamiltonian"}
re_name_dtrIn     = {0: "no", 1: "yes"}
re_name_tranType  = {0: "i-i", 1: "i-o", 2: "o-i", 3: "o-o"}
re_name_dtrNoCnt  = {1: "one", 2: "two", 3: "three"}
re_name_trlCorr   = {-1: 'incorrect', 0: 'correct'}

angleTraj_pd_corr = angleTraj_pd_corr.replace({"trialType": re_name_trialType})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"dtrIn": re_name_dtrIn})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"transType": re_name_tranType})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"dtrNoCnt": re_name_dtrNoCnt})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"trialMark": re_name_trlCorr})
angleTraj_pd_corr["trialCnt"] = sp.stats.zscore(np.log(angleTraj_pd_corr["trialCnt"]))
angleTraj_pd_corr["recencyL"] = sp.stats.zscore(angleTraj_pd_corr["recencyL"])
angleTraj_pd_corr["tgtAng"]   = np.abs(angleTraj_pd_corr["tgtAng"])
                        

In [13]:
timeIdx = np.arange(0, 1.5 + 1/sampleRate, 1/sampleRate) # equivalent to: np.linspace(0, respDur, num=121); 
for iR in range(0, len(timeIdx)):  
    for SubIdx in range(0, subLen):
        iR_sub = angleTraj_pd_corr.loc[
            (angleTraj_pd_corr['subNo'] == SubIdx) & (angleTraj_pd_corr["sampleCnt"] == iR), 
            [
                "subNo",
                "trialCnt",
                "trialType",
                "radiusCon",
                "tgtAng",
                "tgtDist",
                "tgtCumSum",
                "tgtSpd",
                "dtrIn",
                "dtrNoCnt",
                "transType",
                "recencyL",
                "trialMark",
                "clsNameFr", 
                "clsNameTo",
                "nodeIdFr",
                "nodeIdTo", 
                "nodeCatFr",
                "nodeCatTo"
            ],
        ]
        trialMark_col = iR_sub.iloc[:-1, :]      # The previous trials; remove the last trial for each subject
        trialMark_col = trialMark_col['trialMark']
        trialMark_col = trialMark_col.to_numpy() # whether response correct or incorrect in the previous trial
 
        iR_sub = iR_sub.iloc[1:, :]              # The current trials
        iR_sub['trialMarkLast'] = trialMark_col
        
        iR_sub['tgtCumSumZs']   = iR_sub['tgtCumSum']
        iR_sub['tgtCumSumZs']   = sp.stats.zscore(iR_sub['tgtCumSumZs'])
        iR_sub['tgtSpdZs']      = iR_sub['tgtSpd']  
        iR_sub['tgtSpdZs']      = sp.stats.zscore(iR_sub['tgtSpdZs'])
        
        if SubIdx == 0:
            data_iR = iR_sub
        else:
            data_iR = data_iR.append(iR_sub)
        
    path_or_buf = data_behv + "angleCal/" + str(iR) + "_allTrials_" + "data_iR.csv"
    data_iR.to_csv(path_or_buf=path_or_buf)
    

### Split the trials into Learning and Testing phases (with each dataset standardized separately) and store them separately.

#### learning stage

In [14]:
angleTraj_pd_corr = angleTraj_pd_subj
angleTraj_pd_corr = angleTraj_pd_corr.loc[angleTraj_pd_corr["blockNo"] < 3, :]

re_name_trialType = {1: "random", 2: "hamiltonian"}
re_name_dtrIn     = {0: "no", 1: "yes"}
re_name_tranType  = {0: "i-i", 1: "i-o", 2: "o-i", 3: "o-o"}
re_name_dtrNoCnt  = {1: "one", 2: "two", 3: "three"}
re_name_trlCorr   = {-1: 'incorrect', 0: 'correct'}

angleTraj_pd_corr = angleTraj_pd_corr.replace({"trialType": re_name_trialType})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"dtrIn": re_name_dtrIn})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"transType": re_name_tranType})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"dtrNoCnt": re_name_dtrNoCnt})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"trialMark": re_name_trlCorr})
angleTraj_pd_corr["trialCnt"] = sp.stats.zscore(np.log(angleTraj_pd_corr["trialCnt"]))
angleTraj_pd_corr["recencyL"] = sp.stats.zscore(angleTraj_pd_corr["recencyL"])
angleTraj_pd_corr["tgtAng"]   = np.abs(angleTraj_pd_corr["tgtAng"])


In [15]:
timeIdx = np.arange(0, 1.5 + 1/sampleRate, 1/sampleRate) # equivalent to: np.linspace(0, respDur, num=121); 
for iR in range(0, len(timeIdx)):  
    for SubIdx in range(0, subLen):
        iR_sub = angleTraj_pd_corr.loc[
            (angleTraj_pd_corr['subNo'] == SubIdx) & (angleTraj_pd_corr["sampleCnt"] == iR), 
            [
                "subNo",
                "trialCnt",
                "trialType",
                "radiusCon",
                "tgtAng",
                "tgtDist",
                "tgtCumSum",
                "tgtSpd",
                "dtrIn",
                "dtrNoCnt",
                "transType",
                "recencyL",
                "trialMark",
                "clsNameFr", 
                "clsNameTo",
                "nodeIdFr",
                "nodeIdTo", 
                "nodeCatFr",
                "nodeCatTo"
            ],
        ]
        trialMark_col = iR_sub.iloc[:-1, :]      # The previous trials; remove the last trial for each subject
        trialMark_col = trialMark_col['trialMark']
        trialMark_col = trialMark_col.to_numpy() # whether response correct or incorrect in the previous trial

        iR_sub = iR_sub.iloc[1:, :]              # The current trials
        iR_sub['trialMarkLast'] = trialMark_col
        
        iR_sub['tgtCumSumZs']   = iR_sub['tgtCumSum']
        iR_sub['tgtCumSumZs']   = sp.stats.zscore(iR_sub['tgtCumSumZs'])
        iR_sub['tgtSpdZs']      = iR_sub['tgtSpd']  
        iR_sub['tgtSpdZs']      = sp.stats.zscore(iR_sub['tgtSpdZs'])
        
        if SubIdx == 0:
            data_iR = iR_sub
        else:
            data_iR = data_iR.append(iR_sub)
        
    path_or_buf = data_behv + "angleCal/" + str(iR) + "_learning_" + "data_iR.csv"
    data_iR.to_csv(path_or_buf=path_or_buf)
    

#### testing stage

In [16]:
angleTraj_pd_corr = angleTraj_pd_subj
angleTraj_pd_corr = angleTraj_pd_corr.loc[angleTraj_pd_corr["blockNo"] >= 3, :]

re_name_trialType = {1: "random", 2: "hamiltonian"}
re_name_dtrIn     = {0: "no", 1: "yes"}
re_name_tranType  = {0: "i-i", 1: "i-o", 2: "o-i", 3: "o-o"}
re_name_dtrNoCnt  = {1: "one", 2: "two", 3: "three"}
re_name_trlCorr   = {-1: 'incorrect', 0: 'correct'}

angleTraj_pd_corr = angleTraj_pd_corr.replace({"trialType": re_name_trialType})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"dtrIn": re_name_dtrIn})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"transType": re_name_tranType})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"dtrNoCnt": re_name_dtrNoCnt})
angleTraj_pd_corr = angleTraj_pd_corr.replace({"trialMark": re_name_trlCorr})
angleTraj_pd_corr["trialCnt"] = sp.stats.zscore(np.log(angleTraj_pd_corr["trialCnt"]))
angleTraj_pd_corr["recencyL"] = sp.stats.zscore(angleTraj_pd_corr["recencyL"])
angleTraj_pd_corr["tgtAng"]   = np.abs(angleTraj_pd_corr["tgtAng"])


In [17]:
timeIdx = np.arange(0, 1.5 + 1/sampleRate, 1/sampleRate) # equivalent to: np.linspace(0, respDur, num=121); 
for iR in range(0, len(timeIdx)):  
    for SubIdx in range(0, subLen):
        iR_sub = angleTraj_pd_corr.loc[
            (angleTraj_pd_corr['subNo'] == SubIdx) & (angleTraj_pd_corr["sampleCnt"] == iR), 
            [
                "subNo",
                "trialCnt",
                "trialType",
                "radiusCon",
                "tgtAng",
                "tgtDist",
                "tgtCumSum",
                "tgtSpd",
                "dtrIn",
                "dtrNoCnt",
                "transType",
                "recencyL",
                "trialMark",
                "clsNameFr", 
                "clsNameTo",
                "nodeIdFr",
                "nodeIdTo", 
                "nodeCatFr",
                "nodeCatTo"
            ],
        ]
        trialMark_col = iR_sub.iloc[:-1, :]      # The previous trials; remove the last trial for each subject
        trialMark_col = trialMark_col['trialMark']
        trialMark_col = trialMark_col.to_numpy() # whether response correct or incorrect in the previous trial
 
        iR_sub = iR_sub.iloc[1:, :]              # The current trials
        iR_sub['trialMarkLast'] = trialMark_col
        
        iR_sub['tgtCumSumZs']   = iR_sub['tgtCumSum']
        iR_sub['tgtCumSumZs']   = sp.stats.zscore(iR_sub['tgtCumSumZs'])
        iR_sub['tgtSpdZs']      = iR_sub['tgtSpd']  
        iR_sub['tgtSpdZs']      = sp.stats.zscore(iR_sub['tgtSpdZs'])
        
        if SubIdx == 0:
            data_iR = iR_sub
        else:
            data_iR = data_iR.append(iR_sub)
        
    path_or_buf = data_behv + "angleCal/" + str(iR) + "_testing_" + "data_iR.csv"
    data_iR.to_csv(path_or_buf=path_or_buf)
    

### Split the trials into Learning and Testing phases (after standardizing all data together) and store them separately.

#### learning stage

In [ ]:
angleTraj_pd_learning = angleTraj_pd_corr
angleTraj_pd_learning = angleTraj_pd_learning.loc[angleTraj_pd_learning["blockNo"] < 3, :]
timeIdx = np.arange(0, 1.5 + 1/sampleRate, 1/sampleRate) # equivalent to: np.linspace(0, respDur, num=121); 
for iR in range(0, len(timeIdx)):  
    for SubIdx in range(0, subLen):
        iR_sub = angleTraj_pd_learning.loc[
            (angleTraj_pd_learning['subNo'] == SubIdx) & (angleTraj_pd_learning["sampleCnt"] == iR), 
            [
                "subNo",
                "trialCnt",
                "trialType",
                "radiusCon",
                "tgtAng",
                "tgtDist",
                "tgtCumSum",
                "tgtSpd",
                "dtrIn",
                "dtrNoCnt",
                "transType",
                "recencyL",
                "trialMark",
                "clsNameFr", 
                "clsNameTo",
                "nodeIdFr",
                "nodeIdTo", 
                "nodeCatFr",
                "nodeCatTo"
            ],
        ]
        trialMark_col = iR_sub.iloc[:-1, :]      # The previous trials; remove the last trial for each subject
        trialMark_col = trialMark_col['trialMark']
        trialMark_col = trialMark_col.to_numpy() # whether response correct or incorrect in the previous trial

        iR_sub = iR_sub.iloc[1:, :]              # The current trials
        iR_sub['trialMarkLast'] = trialMark_col
        
        iR_sub['tgtCumSumZs']   = iR_sub['tgtCumSum']
        iR_sub['tgtCumSumZs']   = sp.stats.zscore(iR_sub['tgtCumSumZs'])
        iR_sub['tgtSpdZs']      = iR_sub['tgtSpd']  
        iR_sub['tgtSpdZs']      = sp.stats.zscore(iR_sub['tgtSpdZs'])
        
        if SubIdx == 0:
            data_iR = iR_sub
        else:
            data_iR = data_iR.append(iR_sub)
        
    path_or_buf = data_behv + "angleCal/" + str(iR) + "_allTrials_learning_" + "data_iR.csv"
    data_iR.to_csv(path_or_buf=path_or_buf)
    

#### testing stage

In [ ]:
angleTraj_pd_testing = angleTraj_pd_corr
angleTraj_pd_testing = angleTraj_pd_testing.loc[angleTraj_pd_testing["blockNo"] >= 3, :]
timeIdx = np.arange(0, 1.5 + 1/sampleRate, 1/sampleRate) # equivalent to: np.linspace(0, respDur, num=121); 
for iR in range(0, len(timeIdx)):  
    for SubIdx in range(0, subLen):
        iR_sub = angleTraj_pd_testing.loc[
            (angleTraj_pd_testing['subNo'] == SubIdx) & (angleTraj_pd_testing["sampleCnt"] == iR), 
            [
                "subNo",
                "trialCnt",
                "trialType",
                "radiusCon",
                "tgtAng",
                "tgtDist",
                "tgtCumSum",
                "tgtSpd",
                "dtrIn",
                "dtrNoCnt",
                "transType",
                "recencyL",
                "trialMark",
                "clsNameFr", 
                "clsNameTo",
                "nodeIdFr",
                "nodeIdTo", 
                "nodeCatFr",
                "nodeCatTo"
            ],
        ]
        trialMark_col = iR_sub.iloc[:-1, :]      # The previous trials; remove the last trial for each subject
        trialMark_col = trialMark_col['trialMark']
        trialMark_col = trialMark_col.to_numpy() # whether response correct or incorrect in the previous trial
 
        iR_sub = iR_sub.iloc[1:, :]              # The current trials
        iR_sub['trialMarkLast'] = trialMark_col
        
        iR_sub['tgtCumSumZs']   = iR_sub['tgtCumSum']
        iR_sub['tgtCumSumZs']   = sp.stats.zscore(iR_sub['tgtCumSumZs'])
        iR_sub['tgtSpdZs']      = iR_sub['tgtSpd']  
        iR_sub['tgtSpdZs']      = sp.stats.zscore(iR_sub['tgtSpdZs'])
        
        if SubIdx == 0:
            data_iR = iR_sub
        else:
            data_iR = data_iR.append(iR_sub)
        
    path_or_buf = data_behv + "angleCal/" + str(iR) + "_allTrials_testing_" + "data_iR.csv"
    data_iR.to_csv(path_or_buf=path_or_buf)